In [2]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import time
import cv2

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from metadata import *
from data_manager import *

from sklearn.cluster import KMeans
from sklearn.externals import joblib

import random
from collections import defaultdict

from sift_spm_utilities import *
from learning_utilities import *
from annotation_utilities import *

import pandas as pd

Setting environment for Gordon


Cannot import mxnet.


In [3]:
output_dir = '/oasis/projects/nsf/csd395/yuncong/CSHL_SIFT_SPM_features/'
M = 200 # vocabulary size
colors = np.vstack([(0,0,0), np.random.randint(0, 255, (M, 3))])

In [4]:
patch_size = 224
stride = 56

In [8]:
username = 'yuncong'
label_polygons = load_label_polygons_if_exists(stack, username, force=False, 
                                               annotation_rootdir=annotation_midbrainIncluded_rootdir)

In [7]:
indices_map = defaultdict(list)
hists0_allLandmarks = defaultdict(list)
hists1_allLandmarks = defaultdict(list)
hists2_allLandmarks = defaultdict(list)

# for stack in ['MD589', 'MD594', 'MD585']:
for stack in ['MD589']:

    username = 'yuncong'
    label_polygons = load_label_polygons_if_exists(stack, username, force=False, 
                                                   annotation_rootdir=annotation_midbrainIncluded_rootdir)

    image_width, image_height = DataManager.get_image_dimension(stack)
    grid_spec = (patch_size, stride, image_width, image_height)
    
    sample_locations = grid_parameters_to_sample_locations(grid_spec=grid_spec)
        
    first_sec, last_sec = detect_bbox_range_lookup[stack]
    bar = show_progress_bar(first_sec, last_sec)
        
#     for sec in range(first_sec, first_sec+2):
#     for sec in range(first_sec, last_sec+1, 5):
    for sec in [first_sec]:

        bar.value = sec

        labelmap = compute_labelmap(stack, sec)
        
        mask_tb = DataManager.load_thumbnail_mask(stack, sec)
        indices_allLandmarks = locate_patches(grid_spec=grid_spec, mask_tb=mask_tb, polygons=label_polygons.loc[sec].dropna())
        
        for name, indices in indices_allLandmarks.iteritems():
            
            if name not in set(labels_unsided) - {'outerContour'}:
                continue
                
            print name
            
#             indices = indices_allLandmarks_allSections[sec][name]

            num_sample = 20
            indices_sampled = np.random.choice(indices, min(len(indices), num_sample), replace=False)
            sample_locs = sample_locations[indices_sampled]
        
            hists_arr0, hists_arr1, hists_arr2 = compute_spm_histograms(labelmap, sample_locs, patch_size=patch_size, M=M)
            
            hists0_allLandmarks[name].append(hists_arr0)
            hists1_allLandmarks[name].append(hists_arr1)
            hists2_allLandmarks[name].append(hists_arr2)
            
            indices_map[name].append(indices_sampled)

hists0_allLandmarks.default_factory = None
hists1_allLandmarks.default_factory = None
hists2_allLandmarks.default_factory = None

indices_map.default_factory = None

VLL
(20, 16, 200)
(20, 4, 200)
(20, 200)
7n


done in 21.339974 seconds


(20, 16, 200)
(20, 4, 200)
(20, 200)
sp5


done in 21.353806 seconds


(20, 16, 200)
(20, 4, 200)
(20, 200)
5N


...

(20, 16, 200)
(20, 4, 200)
(20, 200)
LRt


done in 21.677761 seconds


(20, 16, 200)
(20, 4, 200)
(20, 200)


done in 21.687864 seconds


KeyboardInterrupt: 

In [ ]:
hists0_allLandmarks = {name: np.concatenate(arrs) for name, arrs in hists0_allLandmarks.iteritems()}
hists1_allLandmarks = {name: np.concatenate(arrs) for name, arrs in hists1_allLandmarks.iteritems()}
hists2_allLandmarks = {name: np.concatenate(arrs) for name, arrs in hists2_allLandmarks.iteritems()}
indices_map = {name: [(i,) + l for i, l in enumerate(chain(*indices))] for name, indices in indices_map.iteritems()}

In [ ]:
for name, arr in hists0_allLandmarks.iteritems():
    bp.pack_ndarray_file(arr, output_dir + '/train/%(name)s_histograms_l0.bp' % {'name': name})

for name, arr in hists1_allLandmarks.iteritems():
    bp.pack_ndarray_file(arr, output_dir + '/train/%(name)s_histograms_l1.bp' % {'name': name})

for name, arr in hists2_allLandmarks.iteritems():
    bp.pack_ndarray_file(arr, output_dir + '/train/%(name)s_histograms_l2.bp' % {'name': name})

In [ ]:
def save_table(data, fn):
    with open(fn, 'w') as f:
        for line in data:
            f.write(' '.join([str(c) for c in line]) + '\n')
                
def load_table(fn):
    with open(fn, 'r') as f:
        lines = f.readlines()
    data = [l.split() for l in lines] 
    return data

In [ ]:
for name, ind_map in indices_map.iteritems():
    print name
    save_table(ind_map, output_dir + '/train/train_indices/indices_map_%s.txt' % name)